## Simple LSTM


In [1]:
import torch 
from torch import nn

In [ ]:
class simple_lstm(nn.Module): 
    def __init__(self, input, hidden_dim) -> None:
        super().__init__()
        self.input_gate = nn.Linear()